# Data Cleaning and EDA (Exploratory Data Analysis)

Cleans and explores the data for use in preprocessing.

In [1]:
# imports
import pandas as pd
import numpy as np

import json

from skimage import io
import skimage
import cv2
from keras.utils import np_utils
import tensorflow as tf
import keras

import glob



In [ ]:
pd.read